In [21]:
# Extracts text from the pdf
from pypdf import PdfReader

reader = PdfReader(r'C:\Users\Rupa Bisht\Desktop\Summarization\pdf\Microsoft_outage.pdf')
# printing number of pages in pdf file 
print(len(reader.pages)) 
  

text =""
for page in reader.pages :
    text += page.extract_text() 

print(text) 

1
On July 19, 2024, Microsoft experienced a significant outage that disrupted various services including 
Azure, Microsoft Teams, Microsoft 365, and Power BI. The issue began in the early hours of the day 
and affected users worldwide, causing considerable inc onvenience for both individual and enterprise 
users.  
The root cause of the outage was identified as a misconfiguration during a sensor update by 
CrowdStrike, a cybersecurity firm. The update inadvertently triggered widespread disruptions across 
multiple Microsoft services. According to CrowdStrike, the probl em was related to a logic bug in the 
sensor configuration that affected the systems' ability to communicate with the Microsoft cloud 
services, leading to a cascade of failures.  
Microsoft's Azure status page indicated that the outage impacted a variety of services, including 
Azure SQL Database, Azure Cache for Redis, and Azure Synapse Analytics, among others. These 
disruptions resulted in users being unable to acces

In [22]:
# using sentence_transformers to make vectors of the extracteed text
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
emb = model.encode(text)

print (emb)

[-1.59424007e-01  3.75841036e-02 -4.57694754e-04 -1.16830736e-01
  4.00701225e-01 -2.64875948e-01 -1.61772296e-01  1.74419731e-01
  3.41628671e-01 -3.83372456e-02  7.71753397e-03  4.63353992e-01
  6.71166480e-02 -6.11041963e-01 -4.35599908e-02  5.03327176e-02
  8.60657990e-02 -3.96891862e-01 -3.54967028e-01  1.48834020e-01
 -1.50047168e-02 -1.96079925e-01 -3.96379054e-01 -6.20594397e-02
  4.53941561e-02  3.85729745e-02 -6.94956332e-02  3.48702781e-02
 -6.51488826e-02  8.26837644e-02  2.48158276e-01  1.95514381e-01
  4.05521095e-01 -9.27455872e-02  2.06734300e-01 -8.78723487e-02
  3.29517394e-01 -2.77191818e-01 -3.04389782e-02 -3.20156455e-01
 -6.17034808e-02 -1.12415120e-01 -2.05513284e-01  5.97660691e-02
  2.54076719e-03  2.38136783e-01 -1.32037193e-01 -2.90916324e-01
 -2.34853387e-01 -3.57011706e-02  3.25749189e-01 -3.69151235e-02
  3.00089687e-01 -4.33982089e-02  1.16678447e-01 -2.73196191e-01
  6.55162632e-02  5.87208807e-01 -1.33893490e-01  3.70689631e-01
  1.02215931e-01 -3.70068

In [23]:
# it makes connection with elastic
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200, 'scheme':'http'}])
es.ping()

True

In [24]:
# if the index with the same name exists [it has no use here(only when you want to delete the index), meanwhile let it remain commented]
# es.indices.delete(index="summarization_pdf",ignore_unavailable=True)

# Query to create the index, it's how the data will get mapped[basically data deifining]
# text [stores the text extracted from the document ]
# text_embedding [stores vectors of the text ]
# run this once [because you can't create an index with same name every time]
# es.indices.create(
#     index = "summarization_pdf",
#     mappings={
#         "properties": {
#                 "text": {
#                     "type": "text"    
#                 },
#                 "username": {
#                     "type": "text"
                    
#                 },
#                 "text_embedding": {
#                     "type": "dense_vector",
#                     "dims": 384
#                 },
#                 "fId": {
#                     "type": "text"
#                 },
#                 "fileName":{
#                     "type":"text"
#                 },
#                 "pageNo": {
#                     "type": "text"
#                 },
#                 "tables": {
#                     "type":"keyword"
#                }
#         }
#     }
# )

In [25]:
#  index [where the data will get stored in the Elasticsearch]
index = "summarization_pdf"
doc={"text_embedding": emb ,
     "text": text}
# elastic call to post data in the elasticsearch index
es.index(index = index, body=doc)

ObjectApiResponse({'_index': 'summarization_pdf', '_id': '40PXO5EBtDsXdsaGfTX2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7, '_primary_term': 4})

In [26]:
# Query from user 
query = "When did the Microdsoft systems were causing trouble ?"
# Embeddings od the query(so that it could first search the document (vectors) in elastic)
query_emb = model.encode(query)



In [27]:
# search Query
search_query = {
    "query": {
        "script_score": {
            "query": {
                "match_all": {}
            },
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'text_embedding') + 1.0",
                "params": {
                    "query_vector": query_emb
                }
            }
        }
    },
    "size": 1 
}

# the search call to elastic
response = es.search(index=index, body=search_query)

In [28]:
#  just iterating the searched text from response(above)

for hit in response['hits']['hits']:
    get_text = hit['_source']['text']
    print(get_text)

On July 19, 2024, Microsoft experienced a significant outage that disrupted various services including 
Azure, Microsoft Teams, Microsoft 365, and Power BI. The issue began in the early hours of the day 
and affected users worldwide, causing considerable inc onvenience for both individual and enterprise 
users.  
The root cause of the outage was identified as a misconfiguration during a sensor update by 
CrowdStrike, a cybersecurity firm. The update inadvertently triggered widespread disruptions across 
multiple Microsoft services. According to CrowdStrike, the probl em was related to a logic bug in the 
sensor configuration that affected the systems' ability to communicate with the Microsoft cloud 
services, leading to a cascade of failures.  
Microsoft's Azure status page indicated that the outage impacted a variety of services, including 
Azure SQL Database, Azure Cache for Redis, and Azure Synapse Analytics, among others. These 
disruptions resulted in users being unable to access 

In [29]:
#  here comes the llm
# calling llm mistral[link : https://api-inference.huggingface.co/models/mistralai/Mixtral-8x7B-Instruct-v0.1] from the hugging face 

import requests
def query_mistral(payload):

    #  configurations to make call to mistral
    API_URL = "https://api-inference.huggingface.co/models/mistralai/Mixtral-8x7B-Instruct-v0.1"
    # obviously I am not sharing the token and too lazy to use dotenv
    API_TOKEN = "hf_________________________________ChL"

    # prepares the headers for http request
    headers = {"Authorization": f"Bearer {API_TOKEN}"}

    # the prompt is sent to llm with other parameters and llm's[mistral here] response is stored in the response variable
    response = requests.post(API_URL, headers=headers, json=payload)

    # just to debug information
    print("-> Request ResponseJson:", response.json()) # returns the response[the generated text] by the model
    print("-> Request Response:", response) # returns the status [like 200 if succeed]
    print("-> Request Payload:", payload) 
    print("-> Payload Size:", len(str(payload)))
    print("-> Response Status Code:", response.status_code)
    print("-> Response Content:", response.content) # returns the response[the generated text] by the model
    print("-> Response Content len:", len(str(response.content)))
    
    try:
        # if any API error occurs it prints error an return NONE
        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code}")
            return None
        
        # So the model returns the text with prompt in the list name 'generated_text' along with the prompt we send,
        #  so hereby wee are abstacting only generated text 
        # the prompt text is stored Under the [INST][/INST] tag, hence the text after [INST][/INST] is the model genrated text which is beig extracted in next few lines

        data = response.json()  # storing the list in data 
        generated_text = data[0]['generated_text'] # get only Data at 0th index
        prompt_index = generated_text.find('[/INST]') # the [/INST] tags were appended

        # if prompt_index is not empty then generated_text = text after [/INST]
        if prompt_index != -1:
            generated_text = generated_text[prompt_index + len('[/INST]'):]
        return generated_text
    except requests.exceptions.RequestException as e:
        print("Request exception:", e)
        return None
    except ValueError as e:
        print("Value error:", e)
        return None

#  Setting the prompt
negResponse = "I'm unable to answer the question based on the information I have."
prompt = f"[INST] You are a helpful Q&A assistant. Your task is to answer this question: {query}.Use only the information from this text: {get_text},Provide answer in as less words as possible,If the answer is not contained in the text, reply with {negResponse}. Response [/INST]"
#  setting token limit, so that prompt stay under the tocken limit of the model
max_new_tokens = 2000
max_token = 28000
input_tokens = max_token - max_new_tokens
model_name='mistralai/Mixtral-8x7B-Instruct-v0.1'


# call to the function that calls the model/llm[mistral]
data = query_mistral({"parameters": {"max_new_tokens": max_new_tokens}, "inputs": prompt})





-> Request ResponseJson: [{'generated_text': "[INST] You are a helpful Q&A assistant. Your task is to answer this question: When did the Microdsoft systems were causing trouble ?.Use only the information from this text: On July 19, 2024, Microsoft experienced a significant outage that disrupted various services including \nAzure, Microsoft Teams, Microsoft 365, and Power BI. The issue began in the early hours of the day \nand affected users worldwide, causing considerable inc onvenience for both individual and enterprise \nusers.  \nThe root cause of the outage was identified as a misconfiguration during a sensor update by \nCrowdStrike, a cybersecurity firm. The update inadvertently triggered widespread disruptions across \nmultiple Microsoft services. According to CrowdStrike, the probl em was related to a logic bug in the \nsensor configuration that affected the systems' ability to communicate with the Microsoft cloud \nservices, leading to a cascade of failures.  \nMicrosoft's Azur

In [30]:
#  the final output
print("You Asked ========>  ",query)
print("Answer =========>",data)

You Asked ========>   When did the Microdsoft systems were causing trouble ?
Answer =========>  On July 19, 2024, Microsoft services like Azure, Microsoft Teams, Microsoft 365, and Power BI were disrupted due to a misconfiguration during a sensor update by CrowdStrike. This caused a logic bug that affected the systems' ability to communicate with Microsoft cloud services, leading to a cascade of failures.
